In [ ]:
# define the keras model
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
import pandas as pd 
import numpy as np
import seaborn as sns
from matplotlib import pyplot
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE, SelectFromModel
from sklearn.inspection import permutation_importance
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from imblearn.combine import SMOTEENN
pd.set_option('display.max_columns', None)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
!wget -cO - https://datahack-prod.s3.amazonaws.com/train_file/train_s3TEQDk.csv > train_data.csv
!wget -cO - https://datahack-prod.s3.amazonaws.com/test_file/test_mSzZ8RL.csv > test_data.csv

--2021-05-30 03:45:12--  https://datahack-prod.s3.amazonaws.com/train_file/train_s3TEQDk.csv
Resolving datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)... 52.219.160.31
Connecting to datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)|52.219.160.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13782835 (13M) [text/csv]
Saving to: ‘STDOUT’

-                   100%[===================>]  13.14M  5.04MB/s    in 2.6s    

2021-05-30 03:45:16 (5.04 MB/s) - written to stdout [13782835/13782835]

--2021-05-30 03:45:16--  https://datahack-prod.s3.amazonaws.com/test_file/test_mSzZ8RL.csv
Resolving datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)... 52.219.66.104
Connecting to datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)|52.219.66.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5696419 (5.4M) [text/csv]
Saving to: ‘STDOUT’

-                   100%[===================>]   5.43M

In [ ]:
# remove ID and Is_Lead column from train data and separate out y_train
train_data = pd.read_csv('train_data.csv')
train_data.drop(['ID'],axis=1,inplace=True)
Y_train = train_data['Is_Lead']
train_data.drop(['Is_Lead'], axis=1, inplace=True)

# remove ID column from test data
test_data= pd.read_csv('test_data.csv')
z1 = test_data.drop(['ID'], axis=1)
dataframe = pd.concat([train_data, z1], axis=0)
print(train_data.shape, train_data.columns)

def model(lr):
    model = Sequential()
    model.add(Dense(256, input_dim=46, activation='relu'))
    model.add(Dropout(.1, input_shape=(2,)))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.1, input_shape=(2,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(.1, input_shape=(2,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(.1, input_shape=(2,)))    
    model.add(Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

(245725, 9) Index(['Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code', 'Vintage',
       'Credit_Product', 'Avg_Account_Balance', 'Is_Active'],
      dtype='object')


In [ ]:
# checking out imbalance dataset
dataframe['Credit_Product'] = dataframe['Credit_Product'].map({'Yes':1 ,'No':0})
train_data = dataframe.iloc[:train_data.shape[0],:]
print("percentage of imbalance data", Y_train.astype(bool).sum(axis=0)/Y_train.shape[0], Y_train[Y_train == 1].shape)
z1 = dataframe.iloc[train_data.shape[0]:,:]
print(train_data.shape, z1.shape)

percentage of imbalance data 0.23720826126767727 (58288,)
(245725, 9) (105312, 9)


In [ ]:
# data fit transformation using onehot encoding and log transformation
cols = ['Region_Code','Occupation','Channel_Code','Is_Active','Gender']
columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(drop='first', sparse=False), cols)])
transformed = pd.concat([pd.DataFrame(columnTransformer.fit_transform(train_data)), train_data.drop(cols,axis=1)], axis=1)
transformed['Avg_Account_Balance'] = np.log(transformed['Avg_Account_Balance'])

# fit imputer on the dataset
imputer = IterativeImputer()
imputer.fit(transformed)
transformed = imputer.transform(transformed)

# Scaling of data
scaler = StandardScaler()
trans_train = scaler.fit_transform(transformed)

# test data transform
test_trans = pd.concat([pd.DataFrame(columnTransformer.transform(z1)), z1.drop(cols,axis=1)], axis=1)
test_trans['Avg_Account_Balance'] = np.log(test_trans['Avg_Account_Balance'])
test_trans = imputer.transform(test_trans)
trans_test = scaler.transform(test_trans)

# handle imbalance dataset using resampling
# sme = SMOTEENN(random_state=42)
# trans_train, Y_train = sme.fit_resample(trans_train, Y_train)
print(trans_test.shape, trans_train.shape, Y_train.shape)

(105312, 46) (245725, 46) (245725,)


In [ ]:
# fit the keras model on the dataset
class_weight = {1: 0.75, 0: 0.25}
model1 = model(0.001)
model1.fit(trans_train, Y_train, epochs=1000, batch_size=200, shuffle=True)

Epoch 1/1000
1229/1229 [==============================] - 7s 5ms/step - loss: 0.3993 - accuracy: 0.8315
Epoch 2/1000
1229/1229 [==============================] - 6s 5ms/step - loss: 0.3645 - accuracy: 0.8542
Epoch 3/1000
1229/1229 [==============================] - 6s 5ms/step - loss: 0.3605 - accuracy: 0.8556
Epoch 4/1000
1229/1229 [==============================] - 6s 5ms/step - loss: 0.3588 - accuracy: 0.8558
Epoch 5/1000
1229/1229 [==============================] - 6s 5ms/step - loss: 0.3566 - accuracy: 0.8564
Epoch 6/1000
1229/1229 [==============================] - 6s 5ms/step - loss: 0.3547 - accuracy: 0.8567
Epoch 7/1000
1229/1229 [==============================] - 6s 5ms/step - loss: 0.3534 - accuracy: 0.8572
Epoch 8/1000
1229/1229 [==============================] - 6s 5ms/step - loss: 0.3527 - accuracy: 0.8574
Epoch 9/1000
1229/1229 [==============================] - 6s 5ms/step - loss: 0.3526 - accuracy: 0.8571
Epoch 10/1000
1229/1229 [==============================] - 7s 5m

In [ ]:
xgb_pred = model1.predict(trans_test)
final_xgb=pd.DataFrame()
final_xgb['Is_Lead'] = xgb_pred.reshape(-1,)
final=pd.DataFrame()
final['ID'] = test_data['ID']
final=final.join(final_xgb)
print(final.info())

print(final)
final.to_csv('prediction.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105312 entries, 0 to 105311
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   ID       105312 non-null  object 
 1   Is_Lead  105312 non-null  float32
dtypes: float32(1), object(1)
memory usage: 1.2+ MB
None
              ID   Is_Lead
0       VBENBARO  0.007459
1       CCMEWNKY  0.941231
2       VK3KGA9M  0.022013
3       TT8RPZVC  0.041632
4       SHQZEYTZ  0.011418
...          ...       ...
105307  DBENJOYI  0.998817
105308  CWQ72DWS  0.639412
105309  HDESC8GU  0.025902
105310  2PW4SFCA  0.175551
105311  F2NOYPPZ  0.062504

[105312 rows x 2 columns]
